In [ ]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
from seaborn import set_style
import seaborn as sns
## This sets the plot style
## to have a grid on a white background
set_style("white")

#For machine learning methods
from sklearn.model_selection import train_test_split

#For neural network
from tensorflow import keras 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img

## Importing the things
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
from keras.utils import to_categorical
import json

import skimage
print(skimage.__version__)

import warnings
warnings.filterwarnings('ignore')

## Checks what is tensorflow running on
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## This imports datasets stored in keras
from keras.datasets import mnist

# Sample of size 15,000 and 100 labels
- Note: for the baseline model we used randomly selected sample of images out of the original data set. For our final model, we use specific group 100 bird species found in the New York area. We compare the relative performace between the two models. 


# 1. Data preprocessing 
- Here I have a new dataset for training. In this case, I have selected 100 randomly chosen bird species out of 400 in total. The selected labels are used for running a query in the original database (birds.csv) to collect all images and data points correspoinding to these 100 labels, in total $15694$ data points or observations. Since these labels were randomly selected the numerical indices are not orderly unlike the entire dataset. This causes a problem with the output neron layer in the feed-forward architecture, with many of the images having unclassified labels. So I have assigned new numerical labels $0 - 99$ to each image. This way we can use tensorflow method (to_categorical) to convert the digits to $[1,0,0,..0]$ format compatible with softmax probability output.    

In [ ]:
birds_db = pd.read_csv('./birds_archive/birds.csv')
birds_db.value_counts('labels')
birds_class = pd.read_csv('./birds_archive/class_dict.csv')
birds_names = pd.read_csv('./birds_archive/birds latin names.csv')

In [ ]:
birds_names_subset = birds_names.sample(100, random_state=42)
birds_names_subset = birds_names_subset.rename(columns={'class':'labels'})
print(birds_names_subset)

birds_db_subset = birds_db[birds_db['labels'].isin(birds_names_subset['labels'])].dropna()
birds_db_subset = birds_db_subset.reset_index()
birds_db_subset

label_dict = {i:birds_names_subset['labels'].values[i] for i in range(len(birds_names_subset))}
labelsDF = pd.DataFrame(label_dict.items(), columns=['label_index','label'])

# 2. Import images
- Import all images corresponding to 100 labels selected above and converted them an array. For this model, we implement scikit-learn tools to split the data into train test split for validation purposes. 

In [ ]:
%%time 
# this piece loads image data into a list and a numpy array 
bird_labels = []
bird_label_num = []
birdImage_list_jpg = []
birdImage_list = []

for indx, filepath, label in zip(range(len(birds_db_subset)), birds_db_subset.filepaths, birds_db_subset.labels):
    try:
        bird_labels.append(label)
        bird_label_num.append(labelsDF.loc[labelsDF['label'] == label, 'label_index'].values[0])
        #print(filepath)
        img = tf.keras.utils.load_img(
                './birds_archive/Data/'+filepath,
                grayscale=False,
                color_mode='rgb',
                target_size=None,
                interpolation='nearest',
                keep_aspect_ratio=False)
        
        # list of bird images in jpg format 
        birdImage_list_jpg.append(img)  
        
        img_arr = np.array(img)
        # List containng all bird images each in array format 
        birdImage_list.append(img_arr)
    except (TypeError, IndexError) as e:
        pass

# Numpy array N x 224x224x3 containing all bird images 
birdImage_arr = np.array(birdImage_list) 

X_train, X_val, y_train, y_val = train_test_split(birdImage_arr, (pd.DataFrame(bird_label_num)[0]).values,
                                                                          test_size=0.15, shuffle=True, random_state=44)
X_train = X_train / 255
X_val = X_val / 255

# 3. Exploring train data

In [ ]:
img = tf.keras.utils.load_img(
                './birds_archive/Data/'+filepath,
                grayscale=False,
                color_mode='rgb',
                target_size=None,
                interpolation='nearest',
                keep_aspect_ratio=False)


i, (im1, im2, im3, im4) = plt.subplots(1, 4, sharey=True)
i.set_figwidth(20) 
im1.imshow(img)  #Original image
im2.imshow(np.array(img)[:, : , 0]) #Red
im3.imshow(np.array(img)[:, : , 1]) #Green
im4.imshow(np.array(img)[:, : , 2]) #Blue
i.suptitle('Original & RGB image channels')
plt.show()
plt.close()

# 4. Convolutional Neural Network: architecture 
- This is baseline model built on two fundamental principles of computer vision:  $\textbf{1. Translational invariance}$ and $\textbf{2. Spatial hiararchy}$. This is a simple model with a few  convolutional (invariance) and pooling (hiararchy) layers. The depth values are chosen based on the complexity of input feature space as well as the number of softmax output neurons. Fiiters and other parameters are kept standard.I also implement regularization cconditions to feed-forward as well as convoluitonal layers in order to address overfitting.   


In [ ]:
from keras.regularizers import l2 

modelB = models.Sequential()
n_species = to_categorical(y_train).shape[1]
# Add convolutional layer
#model.add( layers.Conv3D(32, (3,3,3), activation='relu', input_shape=(224,224,3,1)) )
modelB.add( tf.keras.layers.Conv2D(64, 3, activation='relu', input_shape=(224,224, 3)))
modelB.add( layers.MaxPool2D((2,2), strides=2) ) 
# more layers 
modelB.add( layers.Conv2D(64, (3,3), activation='relu') )
modelB.add( layers.MaxPool2D((2,2), strides=2))
modelB.add( layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)) )

modelB.add( layers.MaxPool2D((2,2), strides=2))
modelB.add( layers.Conv2D(128, (3,3), activation='relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))) 

# .Flatten() will flatten the data for us
modelB.add(layers.Flatten())
modelB.add(layers.Dropout(.5))

## # Now we'll add the fully connected layer >>> feedforward
modelB.add(layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)) )
modelB.add(layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
## Finally an output layer
modelB.add(layers.Dense(n_species, activation='softmax'))

modelB.summary()

In [ ]:
# OPTIONAL cell for loading our saved model

from keras.models import load_model

modelB = load_model('convNet_birds_baseline.h5')
modelB.summary()

# 5. ConvNet Training -  100 labels

In [ ]:
%%time
# Compile the model 

modelB.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
## First make the validation set

# ## Fit the model, and store the training history
## using 100 epochs and a batch_size of 512
n_epoch= 5
historyB = modelB.fit(X_train, to_categorical(y_train), epochs=n_epoch, batch_size=512,
                   validation_data=(X_val, to_categorical(y_val)), callbacks=[callback], verbose=1)

historyB_dict = historyB.history
print(historyB_dict.keys())

# 6. Save model to use later

In [ ]:
modelB.save("convNet_birds_baseline_reg.h5")
print("Saved model to disk")

# 7. Model performance
- we save our array and retrain later and add the improvement to the existing list of epochs

In [ ]:
# run ONLY once
valB_accuracy = []
trainB_accuracy = []

In [ ]:
for tr, val, loss in zip(historyB_dict['accuracy'], historyB_dict['val_accuracy'], historyB_dict['val_loss']): 
    trainB_accuracy.append(tr)
    valB_accuracy.append(val)
    
with open("train_accuracy", "w") as fp:
    json.dump(trainB_accuracy, fp)

with open("validation_accuracy", "w") as fp:
    json.dump(valB_accuracy, fp)

In [ ]:
with open("train_accuracy", "r") as fp:
    trainB_accuracy = json.load(fp)   
with open("validation_accuracy", "r") as fp:
    valB_accuracy = json.load(fp)
    
N = len(valB_accuracy) #n_epoch-1
# Display the metrics
set_style("whitegrid")
plt.figure(figsize=(12,8))
plt.plot(range(1,N+1), trainB_accuracy,'b--^', markersize=5,label='training accuracy')
plt.plot(range(1,N+1), valB_accuracy,'g--o', markersize=5, alpha=0.8, label='validation accuracy')
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)

plt.axhline(y=0.5, color='crimson', ls='--', alpha=1)
plt.axhline(y=np.array(valB_accuracy).max(), color='k', ls='--')

plt.text(1,np.round(np.array(valB_accuracy).max(),2)+0.01, 
         '$Max ~validation ~ accuracy$ = ' +str(np.round(np.array(valB_accuracy).max(),2)), 
         color='k', fontsize=12 )


plt.legend(fontsize=14)
plt.savefig('perfomance_convNetBaseline.pdf')
plt.show()
plt.close()

set_style("white")
# Prediction power
n=np.random.binomial(100,0.5,1)[0]
predicted_bird = modelB.predict(X_val)[n].argmax()
actual_bird = y_val[n]

plt.figure(figsize=(10,10))
plt.imshow(X_val[n], cmap='gray')
plt.text(1,220, 'Predicted: ' + str(labelsDF.loc[labelsDF['label_index'] == predicted_bird, 'label'].values[0]),
         color='yellow', fontsize=14 )

plt.text(1,210, 'Actual: ' + str(labelsDF.loc[labelsDF['label_index'] == actual_bird, 'label'].values[0]),
         color='brown', fontsize=14 )


plt.show()
plt.close()